# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# Kunisch Features from ResNet architectures
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara


## Imports

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Mounting Google Drive

In [15]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    root_dir = 'drive/MyDrive/TesisMV/'
except:
    root_dir = '../'

## Dataset and model selection

In [16]:
#modify only this cell
USE_RN50 = False
SUBCHAPTERS = False
DS_FLAGS = []
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 1
RANDOM_TIMES = 1
ELASTIC_TIMES = 1
GAUSBLUR_TIMES = 1
K = 4

In [17]:
# This cells builds the data_flags variable, that will be used
# to map the requested data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
         'gausblur': GAUSBLUR_TIMES
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = MAP_TIMES.keys()
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

subchapter_str = 'subchapters/' if SUBCHAPTERS else ''
Kfolds = {}

for i in range(0, K):
    print("Fold ", i)
    patterns_dir = os.path.join(root_dir, 'patterns', subchapter_str + data_flags, str(i))
    labels_dir = os.path.join(root_dir, 'labels', subchapter_str + data_flags, str(i))
    # models_path = folder_path + 'models/' + subchapter_str + (f'resnet50_{data_flags}.pth' if USE_RN50 else f'resnet18_{data_flags}.pth')
    # features_path = folder_path + 'features/' + subchapter_str + (f'resnet50_{data_flags}/' if USE_RN50 else f'resnet18_{data_flags}/')
    #rn = 50
    #ep = 65
    #models_path = folder_path + f"models/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}.pth"
    #features_path = folder_path + f"features/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}/"
    rn = 50 if USE_RN50 else 18
    models_path = os.path.join(root_dir, 'models', 'resnet', data_flags, f'resnet{rn}_K0.pth')
    features_dir = os.path.join(root_dir, 'features', 'resnet', data_flags, f'resnet{rn}_K{i}')

    if not (os.path.isdir(patterns_dir) and os.path.isdir(labels_dir)):
        print(patterns_dir)
        print(labels_dir)
        raise FileNotFoundError("""
        No existen directorios de datos para el conjunto de flags seleccionado. 
        Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation.
        """)
    if not (os.path.isfile(models_path)):
        print(models_path)
        raise FileNotFoundError(f"""
        No se encontró modelo para el conjunto de flags seleccionado. 
        Verifique que el modelo exista y, de lo contrario, llame a ResNet Retraining
        """)
                                
    Kfolds[i] = {
        'patterns_dir': patterns_dir,
        'labels_dir': labels_dir,
        'model_path': models_path,
        'features_dir': features_dir
    }
    
    print("--Pattern set encontrado en {}".format(patterns_dir))
    print("--Labels set encontrado en {}".format(labels_dir))
    print("--Modelo encontrado en {}".format(models_path))
    print("--Features a guardar en {}".format(features_dir))


Fold  0
--Pattern set encontrado en ../patterns\base\0
--Labels set encontrado en ../labels\base\0
--Modelo encontrado en ../models\resnet\base\resnet18_K0.pth
--Features a guardar en ../features\resnet\base\resnet18_K0
Fold  1
--Pattern set encontrado en ../patterns\base\1
--Labels set encontrado en ../labels\base\1
--Modelo encontrado en ../models\resnet\base\resnet18_K0.pth
--Features a guardar en ../features\resnet\base\resnet18_K1
Fold  2
--Pattern set encontrado en ../patterns\base\2
--Labels set encontrado en ../labels\base\2
--Modelo encontrado en ../models\resnet\base\resnet18_K0.pth
--Features a guardar en ../features\resnet\base\resnet18_K2
Fold  3
--Pattern set encontrado en ../patterns\base\3
--Labels set encontrado en ../labels\base\3
--Modelo encontrado en ../models\resnet\base\resnet18_K0.pth
--Features a guardar en ../features\resnet\base\resnet18_K3


## Dataset loader

In [18]:
class PatternDataset(Dataset):
    def __init__(self, root_dir, transform=None, build_classification=False, name_cla='output.cla'):
        self.root_dir = root_dir
        self.transform = transform
        self.namefiles = []


        self.classes = sorted(os.listdir(self.root_dir))

        for cl in self.classes:
            for pat in os.listdir(os.path.join(self.root_dir, cl)):
                self.namefiles.append((pat, cl))

        print(f'Files:{len(self.namefiles)}')
        #self.namefiles = sorted(self.namefiles, key = lambda x: x[0])
        
        if build_classification:
            dictClasses = dict()

            for cl in self.classes:
                dictClasses[cl] = []

            for index, (name, cl) in enumerate(self.namefiles):
                print(index, name, cl)
                dictClasses[cl].append((name, index))

            with open(name_cla, 'w') as f:
                f.write('PSB 1\n')
                f.write(f'{len(self.classes)} {len(self.namefiles)}\n')
                f.write('\n')
                for cl in self.classes:
                    f.write(f'{cl} 0 {len(dictClasses[cl])}\n')
                    for item in dictClasses[cl]:
                        f.write(f'{item[1]}\n')
                    f.write('\n')

    def __len__(self):
        return len(self.namefiles)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_name = os.path.join(self.root_dir, self.namefiles[index][1], self.namefiles[index][0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return self.namefiles[index], image


## Funciones auxiliares

In [19]:
def imshow(inp, title = None):
    inp = inp.cpu().detach()
    inp = np.squeeze(inp)
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    plt.show()

def get_vector(model,layer, dim_embedding, x):

  my_embedding = torch.zeros(dim_embedding)

  def copy_data(m,i,o):
    my_embedding.copy_(o.data.squeeze())

  h = layer.register_forward_hook(copy_data)
  model(x)
  h.remove()

  return my_embedding

## Extraction

In [20]:
DEVICE = 0
# 0 3090 (o 1060 en local)
# 1 y 2 2080

In [1]:
random.seed(30)

for i in range(0, K):
    patterns_dir = Kfolds[i]['patterns_dir']
    labels_dir = Kfolds[i]['labels_dir']
    model_path = Kfolds[i]['model_path']
    features_dir = Kfolds[i]['features_dir']
    
    output_train = os.path.join(features_dir, "augmented_train_df.json")
    output_val = os.path.join(features_dir, "val_df.json")
    output_test = os.path.join(features_dir, "test_df.json")

    train_df = pd.read_json(os.path.join(labels_dir, "augmented_train_df.json"), orient='index')
    val_df = pd.read_json(os.path.join(labels_dir, "val_df.json"), orient='index')
    test_df = pd.read_json(os.path.join(labels_dir, "test_df.json"), orient='index')


    device = ('cuda:0' if torch.cuda.is_available() else None)
    if device is None:
        raise Exception("La GPU solicitada no está disponible")

    my_transform = transforms.Compose([ transforms.Resize(224),
                                        #transforms.CenterCrop(224),
                         transforms.ToTensor(),
                         transforms.Normalize(mean=[0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
                        ])

    dataTrain = PatternDataset(root_dir=os.path.join(patterns_dir, 'train'), transform=my_transform)
    dataVal = PatternDataset(root_dir=os.path.join(patterns_dir, 'val'), transform=my_transform)
    dataTest = PatternDataset(root_dir=os.path.join(patterns_dir, 'test'), transform=my_transform)

    loaderTrain = DataLoader(dataTrain, shuffle="False")
    loaderVal = DataLoader(dataVal, shuffle="False")
    loaderTest = DataLoader(dataTest, shuffle="False")

    if USE_RN50:
        model = models.resnet50(pretrained = True)
    else:
        model = models.resnet18(pretrained = True)
    dim = model.fc.in_features

    output_dim = 20 if SUBCHAPTERS else 6
    model.fc = nn.Linear(dim, output_dim)

    model = model.to(device)

    try:
        model.load_state_dict(torch.load(models_path))
    except RuntimeError as e:
        print('Ignoring "' + str(e) + '"')

    layer = model._modules.get('avgpool')

    model.eval()

    #features = []
    features_train = {}
    features_val = {}
    features_test = {}


    for name, img in loaderTrain:
      feat = get_vector(model, layer, dim, img.to(device))
      namefile = name[0][0]
      code, rest = namefile.split('.')
      print(code)
      imshow(img)
      features_train[code] = feat.numpy().tolist()
      #features.append(feat.numpy())

    for name, img in loaderVal:
      feat = get_vector(model, layer, dim, img.to(device))
      namefile = name[0][0]
      code, rest = namefile.split('.')
      #print(code)
      #imshow(img) 
      features_val[code] = feat.numpy().tolist()
      #features.append(feat.numpy())

    for name, img in loaderTest:
      feat = get_vector(model, layer, dim, img.to(device))
      namefile = name[0][0]
      code, rest = namefile.split('.')
      #print(code)
      #imshow(img)
      features_test[code] = feat.numpy().tolist()
    #features = np.vstack(features)
    #print(features.shape)

    os.makedirs(features_dir, exist_ok=True)

    features_train_df = pd.DataFrame.from_dict(features_train, orient='index')
    features_val_df = pd.DataFrame.from_dict(features_val, orient='index')
    features_test_df = pd.DataFrame.from_dict(features_test, orient='index')

    features_train_df.to_json(output_train, orient='index')
    features_val_df.to_json(output_val, orient='index')
    features_test_df.to_json(output_test, orient='index')

    display(features_train_df)

NameError: name 'random' is not defined

In [25]:
display(features_train_df.sort_index())
display(train_df.sort_index())

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
10a,0.296926,1.199218,1.369879,0.643422,0.176883,0.268616,0.109710,0.263788,0.673676,0.283441,...,0.418958,0.490170,0.092686,1.213788,0.641196,0.449828,0.850117,0.075400,1.364796,0.210184
10d,0.457249,0.940334,0.309419,0.032036,0.846819,0.018205,0.049692,0.033167,1.488761,0.269768,...,0.393921,0.734098,0.081450,0.726774,0.023768,1.375203,0.799125,0.005225,0.090596,0.017768
10e,0.423275,1.218758,0.403037,0.038962,0.758215,0.016817,0.002418,0.064993,1.260108,0.266951,...,0.481816,0.545772,0.069476,0.481746,0.107666,1.230064,0.625527,0.009343,0.366649,0.019905
11a,0.350269,1.434289,0.314116,0.011289,1.218893,0.083934,0.024067,0.031605,0.863308,0.152488,...,0.373787,0.185462,0.018175,0.143042,0.078302,0.800978,1.309156,0.012140,0.356008,0.019813
11b,0.497846,0.772536,0.483007,0.004628,2.391656,0.147035,0.075768,0.018046,0.554869,0.163142,...,0.405585,0.120164,0.011635,0.077039,0.063289,0.950535,2.500183,0.005799,0.066605,0.017281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96f,0.842139,0.882934,0.342086,0.271051,1.067486,0.050782,0.001584,0.080643,0.744654,0.634578,...,0.788242,0.578997,0.145819,0.719519,0.250291,0.688715,0.326042,0.114132,0.028726,0.131858
96i,0.821518,0.846151,0.041394,0.213688,0.306110,0.006371,0.034490,0.168134,0.928858,0.847961,...,0.918233,0.983791,0.080408,1.085499,0.138714,1.022310,0.034489,0.087780,0.170908,0.048798
9c,0.503765,0.753490,0.807084,0.657278,0.446944,0.138285,0.144803,0.334821,0.792854,0.534080,...,0.636944,0.749536,0.428479,0.907983,0.540830,0.608013,0.202008,0.203345,0.600078,0.364608
9d,0.409297,0.626461,0.541541,0.067046,2.662688,0.587811,0.081031,0.010341,0.185682,0.112717,...,0.365416,0.096884,0.057707,0.279244,0.120766,0.262041,3.038632,0.089261,0.236700,0.134286


,pendent,teardrop,horizontal,panel,group,vertical,bar,floating,enclosing,shorter,...,light,body,bird,striped,worm,angular,raised,head,bird-seed,long
10a,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10d,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10e,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96f,0,0,0,1,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
96i,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9c,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9d,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
